**Assignment Week 10**

Due: 4/1/2024

Shimon Greengart

First, I get the data using the professor's code from last lecture.

In [ ]:
import pandas as pd

!pip install kaggle --upgrade
print ("writing kaggle")
!echo "{\"username\":\"shimongreengart\",\"key\":\"c659755ac5876b422e520ad3530ab5f5\"}" > kaggle.json
!sudo mkdir -p ~/.kaggle
!sudo cp /content/kaggle.json ~/.kaggle/kaggle.json
print ("got kaggle json copied")
!chmod 600 /root/.kaggle/kaggle.json
!kaggle --version
!kaggle competitions download -c sentiment-analysis-on-movie-reviews

!unzip sentiment-analysis-on-movie-reviews.zip

train_df = pd.read_csv('/content/train.tsv.zip', compression='zip', sep='\t')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.8-py3-none-any.whl size=111962 sha256=1ca1bf77bc1515a5fd6a791f8c1389851a7788f2e422f14f81b832e5b6c6da4e
  Stored in directory: /root/.cache/pip/wheels/b3/86/2d/2df535a84838c858ca91a03805f9c3131573c8e777fd907689
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16
writing kaggle
got kaggle json copied
Kaggle API 1.6.8
  0% 0.00/1.90M [00:00<?, ?B/s]
100% 1.90M/1.90M [00:00<00:00, 74.2MB/s]
Archive:  sentiment-analysis-on-movie-reviews.zip
  inflating: sampleSubmission.csv    
  inflating: test.tsv.zip            
  inflating: train.tsv.zip           


In [ ]:
print(train_df.head())

   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment  
0          1  
1          2  
2          2  
3          2  
4          2  


I heavily used this link when figuring out what to do: https://www.geeksforgeeks.org/nlp-gensim-tutorial-complete-guide-for-beginners/.

First, I took the phrases given and tokenized them. I used gensim's simple_preprocess function to lowercase everything and remove 1-letter words. Then I used the list of tokenized phrases to train a Word2Vec model using gensim's Word2Vec creator.

In [ ]:
import gensim as gs
import numpy as np
import os

train_np = train_df.to_numpy()

all_phrases = train_np[:, 2]

tokenized = []
for phrase in all_phrases:
    tokenized.append(gs.utils.simple_preprocess(phrase))

# Normally, you would use a prebuilt word2vec or the like for your embedding
# But you wanted us to train a new one
w2v_model = gs.models.word2vec.Word2Vec(sentences=tokenized, workers=os.cpu_count())
# If I want, I could have a min_count ignoring rare words
# But I don't know what threshold I would use
# And given that the dataset contains many subsets of each review, I'm not even sure if it would work

# word vector for the word "time", to get visual indication
print(w2v_model.wv['time'])


[-0.58479345  1.8454291   1.3871014   0.20396076  0.11060505 -1.1130906
  1.1975371   0.50944686 -0.26329017  1.5033654   0.5495204  -0.7427096
  2.3862298   0.89145994 -0.44224218  1.5086163   1.4250164  -0.78379273
 -1.4392302  -0.85365295  0.55524343 -1.160308    1.0822506  -1.7084757
 -2.028357   -0.5883192  -1.8655173   1.9735564  -1.9622779   1.0758284
 -1.1275796   0.79752004  2.3454978  -0.14222625 -0.41791037  0.02216911
  0.29475316 -0.5183934  -2.128953   -0.61547214 -0.15247567  2.1783714
 -1.6378458   0.09207394  0.00904999  0.6541906   0.60721916 -0.13557452
 -1.3757231   1.6014705  -0.35181737 -1.2583125  -2.3208618  -0.37793788
  1.4651257   0.37592816  0.6355673   1.860414   -2.0122056   1.256376
 -2.2940717  -1.353127    2.0184145   2.3103702  -0.5965205   0.43871164
 -0.06816463  0.15809098 -0.19589262  0.40652815 -0.43776125 -3.0955908
  0.10209551  2.0006943   0.99855435  1.0352426   1.2796441  -0.5904884
 -1.9830823   0.78886884  1.999134    0.66697234 -1.3769152 

Next I check similar words. You said we could use whatever movie-related words we wanted, so I chose "action", "comedy", "horror", "adventure", and "popcorn". The first four are genres and the last one is fun.

It was interesting. The word most similar to comedy is "quadrangle" (which I'm not even sure how it got into the dataset). The word most similar to horror is "mediocre". Ouch. And the word most similar to "popcorn" is "phony", probably due to complaining about phony popcorn. "Horror" was the 7th-closest word to "action", but not vice versa. Likewise, "comedies" is the 2nd-closest word to "comedy" and the 11th-closest word to "adventure". Anf for some reason, "popcorn" is the 8th-most similar word to "horror", though not vice-versa. And "thriller" was apparently much closer to "comedy" than "adventure" or "action" for reasons I can't determine.

One thing I noticed is that most of these words weren't arbitrary. I suspect that if I spent more time looking at the reviews, the weirder ones would make more sense. I am wondering if some of horror's stranger associations are because sometimes, "horror" refers to the genre, while other times, it refers to how terrible a movie is.

In [ ]:
for word in ["action", "comedy", "horror", "adventure", "popcorn"]:
    print(f"Most similar to {word}:")
    raw_duples = w2v_model.wv.most_similar(word, topn=12)
    print([duple[0] for duple in raw_duples])

Most similar to action:
['animated', 'gangster', 'hong', 'monster', 'kong', 'spy', 'horror', 'abrasive', 'kiddie', 'goofball', 'buddy', 'flick']
Most similar to comedy:
['quadrangle', 'comedies', 'incurably', 'trifle', 'thriller', 'dreaming', 'drama', 'volatile', 'hybrid', 'classic', 'fare', 'psychedelia']
Most similar to horror:
['mediocre', 'monster', 'franchise', 'recent', 'rare', 'worst', 'creepiest', 'popcorn', 'summer', 'baseball', 'thornberrys', 'event']
Most similar to adventure:
['modernized', 'mib', 'fare', 'formula', 'experience', 'suitable', 'year', 'extreme', 'musketeer', 'recycles', 'comedies', 'weepy']
Most similar to popcorn:
['phony', 'solidly', 'jumbled', 'stones', 'evenings', 'baloney', 'types', 'chan', 'slapdash', 'scum', 'overstating', 'marvelous']


Here, I experimented with word subtraction, and these are results I found interesting (or at least humorous). I found that most of the time, if I try (x - y), the result is x, presumably because the subtractor vector wasn't big enough to significantly change the subtractand vector. But I'm skeptical that there is really all that much meaningful here. While it is true that a child without their mother is immediate suspense (the second option, funnily enough, was school), I found enough meaningless differences to make me skeptical that any of them *really* mean anything.

I only used subtraction, not addition, because I found that with addition, the vectors were too big to get any new values. Perhaps with a bigger corpus that uses words in more ways, this would work better.

In [ ]:
word_pairs = [('majestic', 'king'), ('movie', 'story'), ('funny', 'comedy'), ('franchise', 'action'), ('cat', 'person'), ('child', 'mother'), ('war', 'man'), ('man', 'old'), ('woman', 'young')]

for duple in word_pairs:
    diff_vec = w2v_model.wv[duple[0]] - w2v_model.wv[duple[1]]
    result = w2v_model.wv.most_similar(diff_vec, topn=1)[0][0]
    print(f"{duple[0]} - {duple[1]} = {result}")


majestic - king = impair
movie - story = olds
funny - comedy = elizabethans
franchise - action = appreciative
cat - person = mothers
child - mother = suspense
war - man = mystery
man - old = character
woman - young = overrides


In [ ]:
calculations = (w2v_model.wv[duple[0]] - w2v_model.wv[duple[1]] for duple in word_pairs)
for vector in calculations:
    raw_duples = w2v_model.wv.most_similar(vector, topn=5)
    print([duple[0] for duple in raw_duples])

['impair', 'enactments', 'illuminates', 'clinch', 'it']
['olds', 'summer', 'fans', 'wrestling', 'sci']
['elizabethans', 'funny', 'lamentations', 'answers', 'simply']
['appreciative', 'proving', 'dupe', 'swaying', 'bills']
['mothers', 'political', 'tension', 'masturbation', 'violence']
['suspense', 'school', 'crumb', 'urgency', 'center']
['mystery', 'generated', 'fiction', 'processed', 'courtroom']
['character', 'story', 'being', 'man', 'touch']
['overrides', 'says', 'thrilled', 'dungpile', 're']


I was wondering if my model was suffering because their dataset was including many subphrases, so I tried training another version only with the full sentences. But I found that it was actually much worse, from the perspective that the words most similar to my 5 sample words seem completely irrelevant. At least in the first version, they were mostly nouns. Here, the first 3 entries for "action" are prepositions. So, it looks like more data, even repetitive data, was helpful.

In [ ]:
id_set = set()
sentences = []
for row in train_np:
    if row[1] not in id_set:
        id_set.add(row[1])
        sentences.append(row[2])

print(len(sentences))

sent_tokenized = []
for sentence in sentences:
  # the simple_preprocess function returns a list of each sentence
    sent_tokenized.append(gs.utils.simple_preprocess(sentence))

# Normally, you would use a prebuilt word2vec or the like for your embedding
# But you wanted us to train a new one
sent_model = gs.models.word2vec.Word2Vec(sentences=sent_tokenized, workers=os.cpu_count())

for word in ["action", "comedy", "horror", "adventure", "popcorn"]:
    print(f"Most similar to {word}:")
    raw_duples = sent_model.wv.most_similar(word, topn=12)
    print([duple[0] for duple in raw_duples])

8529
Most similar to action:
['between', 'into', 'while', 'dialogue', 'american', 'tale', 'without', 'some', 'comic', 'performances', 'plot', 'three']
Most similar to comedy:
['both', 'some', 'heart', 'full', 'dialogue', 'emotional', 'whose', 'old', 'style', 'despite', 'new', 'drama']
Most similar to horror:
['full', 'mr', 'special', 'off', 'picture', 'being', 'through', 'ultimately', 'comic', 'first', 'tone', 'takes']
Most similar to adventure:
['piece', 'without', 'american', 'yet', 'acting', 'off', 'takes', 'thriller', 'dialogue', 'often', 'human', 'comic']
Most similar to popcorn:
['nearly', 'every', 'series', 'through', 'music', 'great', 'fun', 'being', 'making', 'me', 'our', 'movies']
